In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import statsmodels.formula.api as smf

## Import Data

In [2]:
df = pd.read_csv('data/sample_df.csv')
df.set_index('Month',inplace=True, drop=True)

In [3]:
# df['TV'].index

In [4]:
df.head()

,Volume,TV,Digital,Other,Coverage,RetailPrice,Temp
Month,,,,,,,
2016/01/01,4.679758e+06,24727,4867,0,0.221294,61.822051,6.222581
2016/02/01,5.612667e+06,102245,19383,11877,0.219405,62.132821,7.047126
2016/03/01,7.081166e+06,111393,16725,11987,0.219454,61.820769,10.459140
2016/04/01,8.366490e+06,64051,18815,0,0.217712,61.524615,15.961111
2016/05/01,1.239568e+07,134791,26876,0,0.213228,61.276410,20.646237


## Descriptive Analysis

## Adstock Transformation

### Apply Adstock

In [5]:
#lag and decay and power
def to_adstock(media,lag,decay,alpha):
    
    n = len(media)
    with_AS = np.zeros(n)
    
    for i in range(int(lag),n):
        with_AS[i] = media[i-lag]**(alpha/100) + (decay/100)*with_AS[i-1]
        
    return (with_AS)

## Modelling

### Model fitting

#####   Left: name the pd.Series with Column names following the rule
1. TV_1L: TV_xL x as the number of Lags
2. TV_AS60: AS means astock, this means taking decay of 60% from previous one
3. TV_Alpha90: means taking power curve transformation and, x**aplha, alpha = 0.9

In [6]:
def fit_model1(data,a,b,c,d,e,f,g,h,i):
    
    sales=data['Volume']
    
    tv_ads=to_adstock(data['TV'],a,b,c)
    digital_ads=to_adstock(data['Digital'],d,e,f)
    other_ads=to_adstock(data['Other'],g,h,i)

    temp=data['Temp']
    coverage=data['Coverage']
    retailprice=data['RetailPrice']
    
    x_ad=pd.concat([pd.Series(tv_ads,index=data['TV'].index,name= 'TV_{a}L_{b}AS_{c}Alpha'.format(a=a,b=b,c=c)),
                    pd.Series(digital_ads,index=data['TV'].index,name='Digital_{d}L_{e}AS_{f}Alpha'.format(d=d,e=e,f=f)),
                    pd.Series(other_ads,index=data['TV'].index,name='Other_{g}L_{h}AS_{i}Alpha'.format(g=g,h=h,i=i)),
                    temp,coverage,retailprice,sales], axis=1, join_axes=[data['TV'].index])

    selected=['TV_{a}L_{b}AS_{c}Alpha'.format(a=a,b=b,c=c),
              'Digital_{d}L_{e}AS_{f}Alpha'.format(d=d,e=e,f=f),
              'Other_{g}L_{h}AS_{i}Alpha'.format(g=g,h=h,i=i)]
    
    model=forward_selected(x_ad, 'Volume', selected)


    return model

In [7]:
def forward_selected(data, response, selected):
    remaining = set(data.columns)
    remaining.remove(response)
    for p in selected:
        remaining.remove(p)
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [8]:
def model(data):
    # Run OLS regression, print summary and return results
    
    lag_in =1
    decay_in= -10
    power_in = - 10
    
    tv_lag = list(np.arange(1,4, lag_in))
    tv_decay = list(np.arange(90, 80, decay_in))
    tv_power = list (np.arange(90, 80, power_in))
    digital_lag = list(np.arange(1,4, lag_in))
    digital_decay = list(np.arange(90, 80, decay_in))
    digital_power = list (np.arange(90, 80,power_in))
    other_lag = list(np.arange(1,4, lag_in))
    other_decay = list(np.arange(90, 80, decay_in))
    other_power = list (np.arange(90, 80,power_in))
    final=[]
    for a in tv_lag:
        for b in tv_decay:
            for c in tv_power:
                for d in digital_lag:
                    for e in digital_decay:
                        for f in digital_power:
                            for g in other_lag:
                                for h in other_decay:
                                    for i in other_power:
                                        train_model=fit_model1(data,a,b,c,d,e,f,g,h,i)
                                        final.append(train_model)
    result=best_model(final)
    return result

## 我的思路
1．model　method里面call fit_model1把所有的模型组全部放到final这个list里面
2. 每个变量参数组合算一个adjusted R2,把最小的保留

In [9]:
def best_model(final):
    best= final[0]
    current_score = best.rsquared_adj
    for i in final:
        score=i.rsquared_adj
        if score > current_score:
            best=i
    return best

In [13]:
x = model(df)

In [14]:
print(x.model.formula)

Volume ~ TV_3L_90AS_90Alpha + Digital_3L_90AS_90Alpha + Other_3L_90AS_90Alpha + Temp + Coverage + 1


In [15]:
x.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Volume   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     23.78
Date:                Thu, 21 Feb 2019   Prob (F-statistic):           2.19e-08
Time:                        21:00:40   Log-Likelihood:                -464.04
No. Observations:                  29   AIC:                             940.1
Df Residuals:                      23   BIC:                             948.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -1.431e+07   7.62e+06     -1.878      0.073   -3.01e+07    1.45e+06
TV_3L_90AS_90Alpha        -38.3567     17.118     -2.241      0.035     -73.767      -2.946
Digital_3L_90AS_90Alpha    91.0699     52.387      1.738      0.096     -17.301     199.441
Other_3L_90AS_90Alpha      95.5784    126.822      0.754      0.459    -166.772     357.929
Temp                     4.228e+05   9.82e+04      4.304      0.000     2.2e+05    6.26e+05
Coverage                 7.771e+07    3.7e+07      2.099      0.047    1.11e+06    1.54e+08
==============================================================================
Omnibus:                        0.052   Durbin-Watson:                   1.536
Prob(Omnibus):                  0.974   Jarque-Bera (JB):                0.256
Skew:                           0.049   Prob(JB):                        0.880
Kurtosis:                       2.551   Cond. No.                     2.09e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
pred_x = x.predict()

##### Check MAPE

In [17]:
from sklearn.utils.validation import check_array

In [18]:
def mean_absolute_percentage_error(y_true, y_pred): 
    MAPE = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return MAPE

In [19]:
MAPE = mean_absolute_percentage_error(df['Volume'],pred_x)

In [20]:
MAPE

20.199159466432068